In [5]:
import gensim
from gensim.models import FastText
from gensim.test.utils import common_texts

print(common_texts)


model_ft = FastText(size=5, min_count=1)
model_ft.build_vocab(sentences=common_texts)
model_ft.train(sentences=common_texts, total_examples=len(common_texts), epochs=10)
model_ft

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [21]:
#dir(model_ft.vocabulary)
#dir(model_ft.wv.vocab)#['computer']
model_ft.save("test.model")

In [15]:
model_ft.wv.vocab

{'computer': <gensim.models.keyedvectors.Vocab at 0x11d5858d0>,
 'eps': <gensim.models.keyedvectors.Vocab at 0x11d585518>,
 'graph': <gensim.models.keyedvectors.Vocab at 0x11d5854a8>,
 'human': <gensim.models.keyedvectors.Vocab at 0x11d585978>,
 'interface': <gensim.models.keyedvectors.Vocab at 0x11d585908>,
 'minors': <gensim.models.keyedvectors.Vocab at 0x11d585470>,
 'response': <gensim.models.keyedvectors.Vocab at 0x11d5855c0>,
 'survey': <gensim.models.keyedvectors.Vocab at 0x11d585550>,
 'system': <gensim.models.keyedvectors.Vocab at 0x11d5854e0>,
 'time': <gensim.models.keyedvectors.Vocab at 0x11d585940>,
 'trees': <gensim.models.keyedvectors.Vocab at 0x11d5855f8>,
 'user': <gensim.models.keyedvectors.Vocab at 0x11d585588>}

In [20]:
model_ft.wv["time"].tolist()

[-0.035001490265131,
 -0.00809740275144577,
 0.01750566065311432,
 -0.03750506788492203,
 0.03730582818388939]

In [29]:
def gensim_FastText_model_to_textFile(modelPath):
    
    # load FastText_model
    model = FastText.load(modelPath)
    
    with open('out.txt', 'w') as f:
        f.write(str(len(model.wv.vocab.keys()))+" "+str(len(model.wv[list(model.wv.vocab.keys())[0]]))+"\n")
        for word in model.wv.vocab.keys():
            f.write(word+" ")
            for value in model.wv[word]: f.write(str(value)+" ")
            f.write("\n")
            #print(word, model.wv[word])
        
    
gensim_FastText_model_to_textFile("test.model")

human [-0.01132756  0.01409738  0.04774577  0.01412783  0.00770183]
computer [-0.00130168 -0.01909505  0.01059208  0.00423354  0.02483471]
interface [ 0.00099925  0.0421431   0.00959919  0.0031688  -0.00827794]
time [-0.03500149 -0.0080974   0.01750566 -0.03750507  0.03730583]
minors [ 0.01089878  0.04642535 -0.03265515 -0.00886882 -0.00991796]
graph [-0.03078807  0.02367379  0.03394236 -0.03043788 -0.04659058]
user [-0.09567413  0.00376089 -0.04554972  0.01639676 -0.00488534]
eps [ 0.09038458  0.05576516 -0.06586993  0.06147852 -0.02780395]
survey [-0.03567455  0.03603553  0.01328478  0.00293961 -0.02007984]
system [ 0.0260949  -0.00028982 -0.02077642 -0.0085357   0.03252714]
response [-0.0417905  -0.00754026 -0.04451792  0.01676009  0.00408636]
trees [-0.050333   -0.00055211 -0.02827794  0.02073353  0.04018432]
